In [2]:
from transformers import GitVisionConfig, GitVisionModel

# Initializing a GitVisionConfig with microsoft/git-base style configuration
configuration = GitVisionConfig()

# Initializing a GitVisionModel (with random weights) from the microsoft/git-base style configuration
model = GitVisionModel(configuration)

# Accessing the model configuration
configuration = model.config

/home/alexzigma/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [88]:
from transformers import GitProcessor, AutoImageProcessor, AutoTokenizer, GitModel, GitConfig, GitForCausalLM, AutoModelForCausalLM
from PIL import Image
import requests

image_processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

processor = GitProcessor(image_processor, tokenizer)
model = GitForCausalLM(GitConfig())
# model = AutoModelForCausalLM.from_pretrained("microsoft/git-base-coco")

url = "http://images.cocodataset.org/val2017/000000039769.jpg"

image = Image.open(requests.get(url, stream=True).raw)


In [64]:
inputs = processor(images=image, return_tensors="pt").pixel_values

res = model.generate(pixel_values=inputs)
generated_caption = processor.batch_decode(res, skip_special_tokens=True)[0]
print(generated_caption)

In [112]:
import pandas as pd

captions = pd.read_csv('archive/YouCook.csv', index_col=0)

In [162]:
from transformers import  GitForCausalLM
GitForCausalLM().train()

TypeError: GitForCausalLM.__init__() missing 1 required positional argument: 'config'

In [151]:
import json

# captions = [{"file_name": "ronaldo.jpeg", "text": "Ronaldo with Portugal at the 2018 World Cup"},
# {"file_name": "messi.jpeg", "text": "Messi with Argentina at the 2022 FIFA World Cup"},
# {"file_name": "zidane.jpeg", "text": "Zinédine Zidane pendant la finale de la Coupe du monde 2006."},
# {"file_name": "maradona.jpeg", "text": "Maradona after winning the 1986 FIFA World Cup with Argentina"},
# {"file_name": "ronaldo_.jpeg", "text": "Ronaldo won La Liga in his first season and received the Pichichi Trophy in his second."},
# {"file_name": "pirlo.jpeg", "text": "Pirlo with Juventus in 2014"},]


# path to the folder containing the images
root = "archive/train/"

# add metadata.jsonl file to this folder
with open(root + "metadata.jsonl", 'w') as f:
    for i, j in captions.iterrows():
        f.write(json.dumps({'file_name': j[0], 'text': j[1]}) + "\n")


In [10]:
import torchvision.datasets as dset
import torchvision.transforms as transforms
cap = dset.CocoCaptions(root = 'dir where images are',
                        annFile = 'json annotation file',
                        transform=transforms.PILToTensor())

print('Number of samples: ', len(cap))
img, target = cap[3] # load 4th sample

print("Image Size: ", img.size())
print(target)


TypeError: CocoDetection.__init__() missing 2 required positional arguments: 'root' and 'annFile'

In [160]:
from datasets import load_dataset 

dataset = load_dataset("imagefolder", data_dir=root, split="train")

Resolving data files: 100%|██████████| 89/89 [00:00<00:00, 227756.59it/s]


Extracting data files: 100%|██████████| 88/88 [00:00<00:00, 2863.41it/s]
                                                        

Dataset imagefolder downloaded and prepared to /home/alexzigma/.cache/huggingface/datasets/imagefolder/default-e70151ddb61a5ce4/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f. Subsequent calls will reuse this data.


In [161]:
dataset

Dataset({
    features: ['image', 'Unnamed: 0', 'sentence'],
    num_rows: 0
})

In [ ]:
example = dataset[0]
image = example["image"]
width, height = image.size
display(image.resize((int(0.3*width), int(0.3*height))))


In [ ]:
example["text"]

In [ ]:
from torch.utils.data import Dataset

class ImageCaptioningDataset(Dataset):
    def __init__(self, dataset, processor):
        self.dataset = dataset
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]

        encoding = self.processor(images=item["image"], text=item["text"], padding="max_length", return_tensors="pt")

        # remove batch dimension
        encoding = {k:v.squeeze() for k,v in encoding.items()}

        return encoding

In [ ]:
from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained("microsoft/git-base")

In [ ]:
train_dataset = ImageCaptioningDataset(dataset, processor)

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=2)


In [ ]:
import torch

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

history = []

model.train()

for epoch in range(50):
  print("Epoch:", epoch)
  for idx, batch in enumerate(train_dataloader):
    input_ids = batch.pop("input_ids").to(device)
    pixel_values = batch.pop("pixel_values").to(device)

    outputs = model(input_ids=input_ids,
                    pixel_values=pixel_values,
                    labels=input_ids)
    
    loss = outputs.loss

    history.append(loss)
    print("Loss:", loss.item())

    loss.backward()

    optimizer.step()
    optimizer.zero_grad()
